**Prerequisites**

* Basic knowledge of Python
* Installed `cognite-neat` as a Python package, see [Installation](../installation.html#python-package)

## Introduction

Even though `neat` is made with a domain expert in mind, it can also be used as a Python package. This is a good way of becoming familiar with `neat` if you have some basic Python knowledge. 

In this quickstart, we will show how `neat` can be used to create a data model in Cognite Data Fusion (CDF) from some `JSON` data.

We will start with importing the `importers` and `exporters` from the `rules` modules in `neat`.

In [1]:
from cognite.neat.rules import importer, exporter

An *importer* is a class used to import a data model into `neat`'s internal format for data models, while an *exporter* is a class that is used to export a data model from `neat`'s internal format to a specific output format.

All available importers and exporters can be found in these two modules. In this quickstart, we will use the `ArbitraryJSONImporter` to create a data model in DMS (Data Model Storage) format that can be sent to CDF. This is a type of exporters that infers the data model from a `JSON` string.

First, we will start by getting familiar with the `JSON` data 

## Import Data Model

In [2]:
from tests.data import CAPACITY_BID_JSON

In [3]:
json_string = CAPACITY_BID_JSON.read_text()

print(json_string[:500])

As we see in the print out above, we have some data given in a `JSON` format.

In [4]:
json_importer = importer.ArbitraryJSONImporter(json_string)

rules = json_importer.to_rules()

In the code line above, we import the `JSON` data and covert it to `rules` which is the internal format for data models in `neat`. We can now inspect the rules.

In [5]:
rules.classes

In [6]:
from IPython.display import display, Markdown

for class_id, properties in rules.properties.groupby("class_id").items():
    display(Markdown(f"### {class_id}"))
    display(properties)
    break

In [7]:
display(Markdown(f"We can see that `neat` successfully identified {len(rules.classes)} classes and {len(rules.properties)} properties."))

## Export Data Model

Now we can export these rules to a data model in DMS format which we can write to CDF.

In [8]:
from cognite.client.data_classes.data_modeling import DataModelId

In [9]:
dms_exporter = exporter.DMSExporter(rules, DataModelId("mySpace", "CapacityBid", "1"))

In [10]:
dms_schema = dms_exporter.export()

In [11]:
dms_schema.data_model

In [12]:
dms_schema.containers

These containers can now be written to CDF using the Cognite Python SDK.

```python
from cognite.client import CogniteClient

client = CogniteClient()

client.data_modeling.containers.apply(dms_schema.containers)
client.data_modeling.data_models.apply(dms_schema.data_model)
``` 


We can inspect the data model in CDF's UI Fusion:

![Capacity Bid Data Model](../artifacts/figs/bid_capacity_data_model.png)


## Learn More

You can easily inspect the available importers and exporters in `neat` by importing them as above and using the `.` notation to see the available classes and methods.

The documentation of them are available at [Importers](../api/rules/importers.html) and [Exporters](../api/rules/exporters.html).